In [132]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path

from concurrent.futures import ThreadPoolExecutor, wait

In [120]:
# Парсим через pandas новости и рейтинги

def get_info(page=1):
    url = f'https://news.ycombinator.com/?p={page}'
    
    responce = requests.get(url)
    
    page = pd.read_html(responce.text)[2]
    page.drop([1], axis=1, inplace=True)
    page.rename(columns={0: 'index', 2: 'news'}, inplace=True)

    page = page[(~page['news'].isna()) & (page['news'] != 'More')]

    points = pd.DataFrame(
        page['news'][page['index'].isna()].reset_index(drop=True))
    points.rename(columns={'news':'points'}, inplace=True)

    points['contains_points'] = points['points'].str.contains("points")
    points['points'] = points['points'].str.split(' ')

    def set_points(row):
        if row['contains_points']:
            return row[0][0]
        else:
            return 0

    points['points'] = points.apply(set_points, axis=1)


    news = pd.DataFrame(page['news'][~page['index'].isna()].reset_index(drop=True))

    result = pd.concat([news, points['points']], axis=1)
    
    return result




news = pd.DataFrame(columns=['news', 'points'])
for i in range(9):
    news = pd.concat([news, get_info(i+1)])

---

In [127]:
url_proger = 'https://tproger.ru/quiz/real-programmer'

In [134]:
driver = webdriver.Chrome()
driver.get(url_proger)

driver.maximize_window()
driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")

element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.XPATH, '//*[@id="649c4c45d91e950aa12828ce"]'))
)

element.click()


# driver.close()
# driver.quit()